In [1]:
import datasets
import pandas as pd

In [2]:
semcor = datasets.load_dataset("thesofakillers/SemCor")

Using custom data configuration thesofakillers--SemCor-d701cc7e0f131929
Reusing dataset csv (/Users/thesofakillers/.cache/huggingface/datasets/thesofakillers___csv/thesofakillers--SemCor-d701cc7e0f131929/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
brown1 = semcor["brown1"].to_pandas()
brown1 = brown1.replace(to_replace="None", value=None)

brown2 = semcor["brown2"].to_pandas()
brown2 = brown2.replace(to_replace="None", value=None)

In [4]:
brown1

,tagfile,pnum,snum,tag,lemma,lexsn,wnsn,value,cmd,dc,ot,pn,pos,rdf,sep
0,br-j54,1,1,wf,None,None,None,Whenever,ignore,None,None,None,WRB,None,None
1,br-j54,1,1,wf,artist,1:18:00::,1,artists,done,None,None,None,NN,None,None
2,br-j54,1,1,punc,None,None,None,",",None,None,None,None,None,None,None
3,br-j54,1,1,wf,None,None,None,indeed,done,None,notag,None,RB,None,None
4,br-j54,1,1,punc,None,None,None,",",None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229353,br-j19,25,98,wf,value,1:09:00::,1,value,done,None,None,None,NN,None,None
229354,br-j19,25,98,punc,None,None,None,*,None,None,None,None,None,None,None
229355,br-j19,25,98,punc,None,None,None,*,None,None,None,None,None,None,None
229356,br-j19,25,98,wf,None,None,None,f,done,None,notag,None,NN,None,None


In [5]:
def show_best_candidates(df, top_n=20):
    # most frequent lemmas
    most_frequent = df.lemma.value_counts()
    # senses and pos per lemma
    senses_and_pos_per_lemma = df.groupby("lemma").agg(
        {"lexsn": "nunique", "pos": "nunique"}
    )
    # join
    freq_lemmas = pd.concat([most_frequent, senses_and_pos_per_lemma], axis=1)
    # filter by lemmas with at least 2 senses. Show top n
    filtered = freq_lemmas[(freq_lemmas.lexsn > 1) & (freq_lemmas.pos > 1)][:top_n]
    filtered = filtered.rename(
        columns={"lemma": "lemma count", "lexsn": "sense count", "pos": "pos count"},
        inplace=False,
    )
    filtered.index.name = "Most frequent lemmas"
    return filtered

In [6]:
brown1_best_candidates = show_best_candidates(brown1)

In [7]:
brown1_best_candidates

,lemma count,sense count,pos count
Most frequent lemmas,,,
be,4712,10,2
group,838,3,3
location,617,3,2
only,344,5,2
more,338,6,3
one,335,6,2
see,327,21,2
two,304,2,2
then,293,4,2


In [8]:
brown2_best_candidates = show_best_candidates(brown2)

In [9]:
# how to encode
brown1.lexsn.astype("category").cat.codes

0         -1
1         97
2         -1
3         -1
4         -1
          ..
229353    46
229354    -1
229355    -1
229356    -1
229357    -1
Length: 229358, dtype: int16

In [10]:
# how to group into sentences
brown1.groupby(["tagfile", "pnum", "snum"]).agg({"lemma": list, "value": list})

lemma  \
tagfile pnum snum                                                      
br-a01  1    1     [None, group, say, friday, None, investigation...   
        2    2     [None, jury, far, say, None, term, end, presen...   
        3    3     [None, september, october, term, jury, None, N...   
        4    4     [None, only, None, relative, handful, None, su...   
        5    5     [None, jury, say, None, None, find, None, None...   
...                                                              ...   
br-r09  8    83                        [compassionately, None, None]   
        9    84                                       [person, None]   
        10   85    [None, door, None, None, None, train, slide, s...   
             86    [None, be, None, living, doll, None, None, mis...   
             87    [None, None, None, be, able, None, gauge, None...   

                                                               value  
tagfile pnum snum                                                     
br-a01  1    1     [The, Fulton_County_Grand_Jury, said, Friday, ...  
        2    2     [The, jury, further, said, in, term, end, pres...  
        3    3     [The, September, October, term, jury, had, bee...  
        4    4     [``, Only, a, relative, handful, of, such, rep...  
        5    5     [The, jury, said, it, did, find, that, many_of...  
...                                                              ...  
br-r09  8    83                          [compassionately, yours, ,]  
        9    84                                  [S._J._Perelman, .]  
        10   85    [The, doors, of, the, D, train, slid, shut, ,,...  
             86    [She, was, a, living, doll, and, no, mistake, ...  
             87    [From, what, I, was, able, to, gauge, in, a, s...  

[11182 rows x 2 columns]

In [11]:
brown1_best_candidates

,lemma count,sense count,pos count
Most frequent lemmas,,,
be,4712,10,2
group,838,3,3
location,617,3,2
only,344,5,2
more,338,6,3
one,335,6,2
see,327,21,2
two,304,2,2
then,293,4,2


In [12]:
brown1[brown1.lemma == "location"].pos.unique()

array(['NNP', 'NN'], dtype=object)